In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

c:\Users\pulus\anaconda3\envs\sql_llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def load_job_data(filepath):
    df = pd.read_excel(filepath, nrows=100000)
    
    # Explicitly cast columns to string dtype where applicable
    string_columns = [
        'id',
        'title',
        'company',
        'date_posted',
        'is_remote',
        'interval',
        'min_amount',
        'max_amount',
        'company_industry',
        'emails',
        'location', 'job_type', 'currency']
    df[string_columns] = df[string_columns].astype(str)
    
    df.fillna('', inplace=True)
    
    # Create a combined text field for semantic search
    df['search_text'] = df.apply(lambda row: (
        f"Title: {row['title']}. "
        f"Company: {row['company']}. "
        f"Location: {row['location']}. "
        f"Type: {row['job_type']}. "
        f"Salary: {row['min_amount']}-{row['max_amount']} {row['currency']}. "
        f"Remote: {'Yes' if row['is_remote'] else 'No'}. "
    ), axis=1)
    
    return df

In [4]:
class JobSearchAssistant:
    def __init__(self):
        # Semantic search model
        self.embedder = SentenceTransformer('all-MiniLM-L6-v2')
        
        # Phi-2 for natural language responses
        self.llm_tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")
        self.llm_model = AutoModelForCausalLM.from_pretrained(
            "microsoft/phi-2",
            torch_dtype=torch.float16,
            device_map="auto"
        )
        
    def embed_text(self, text):
        return self.embedder.encode(text)
    
    def generate_response(self, prompt, context=""):
        full_prompt = f"""
        Context about available jobs:{context}
        Question: {prompt}
        Provide a helpful, detailed answer based on the job listings:
        """
        
        # Truncate the full_prompt to fit within the model's max length
        max_input_length = self.llm_tokenizer.model_max_length
        truncated_prompt = full_prompt[:max_input_length]
        print(f"Truncated prompt length: {len(truncated_prompt)}")
        inputs = self.llm_tokenizer(truncated_prompt, return_tensors="pt").to(self.llm_model.device)
        
        outputs = self.llm_model.generate(
            **inputs, 
            max_length=592,
            # max_new_tokens=1000, 
            temperature=0.7, 
            do_sample=True
        )
        
        return self.llm_tokenizer.decode(outputs[0], skip_special_tokens=True)


In [5]:
def find_relevant_jobs(df, query, embedder, top_k=5):
    query_embed = embedder.embed_text(query)
    job_embeddings = embedder.embed_text(df['search_text'].tolist())
    
    # Calculate similarity scores
    scores = np.dot(query_embed, np.array(job_embeddings).T)
    top_indices = np.argsort(scores)[-top_k:][::-1]
    
    return df.iloc[top_indices]


In [6]:
def format_job_results(jobs_df):
    results = []
    for _, job in jobs_df.iterrows():
        results.append(
            f"• {job['title']} at {job['company']}\n"
            f"  - Location: {job['location']} {'(Remote)' if job['is_remote'] else ''}\n"
            f"  - Salary: {job['min_amount']}-{job['max_amount']} {job['currency']}\n"
            f"  - Type: {job['job_type']}\n"
            f"  - Posted: {job['date_posted']}\n"
        )
    return "\n\n".join(results)


In [11]:
def main():
    # Initialize
    df = load_job_data("./all_jobs.xlsx")
    assistant = JobSearchAssistant()
    
    print("Job Search Assistant initialized. Enter your questions about job openings.")
    
    while True:
        try:
            # Get user query
            query = input("\nWhat would you like to know? (or 'quit' to exit)\n> ")
            if query.lower() in ['quit', 'exit']:
                break
                
            # Find relevant jobs
            relevant_jobs = find_relevant_jobs(df, query, assistant)
            context = format_job_results(relevant_jobs)
            
            # Generate natural language response
            response = assistant.generate_response(query, context)
            
            # Display results
            print("\nHere are the most relevant job openings:")
            print(response.split("Answer:")[-1].strip())
            
        except Exception as e:
            print(f"Error: {e}")


In [13]:
main()

C:\Users\pulus\AppData\Local\Temp\ipykernel_4440\1698546704.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 28.48it/s]
Some parameters are on the meta device because they were offloaded to the disk and cpu.


Job Search Assistant initialized. Enter your questions about job openings.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Truncated prompt length: 1056


KeyboardInterrupt: 